# Individual Assingment
# Models Benchmark

### Andres Olivera

In [ ]:
# Models 

# Ensemble
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, BaggingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

# Modeling
from sklearn.model_selection import GridSearchCV    # Cross Validation and Tunning
from sklearn.model_selection import train_test_split
import random

# Metrics
from sklearn.metrics import roc_curve, auc, confusion_matrix, roc_auc_score, accuracy_score, precision_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# Graphs
import matplotlib.pyplot as plt    #PLOT

# Data prep
import numpy as np
import pandas as pd
from sklearn.preprocessing import scale    #Standarize
from sklearn.decomposition import PCA      #Feature Reduction
from imblearn.over_sampling import SMOTE   #Oversampling
from sklearn.feature_selection import RFE  # Feature selection 

In [ ]:
#Reading Data
test = pd.read_csv('C:/Users/aolivera/OneDrive - IESEG/IESEG/Courses/Second Semester/SL&ML/Kaggle/bank_mkt_test.csv')
train = pd.read_csv('C:/Users/aolivera/OneDrive - IESEG/IESEG/Courses/Second Semester/SL&ML/Kaggle/bank_mkt_train.csv')

#### First Approach ===> Dummies, Oversampling, Scaling and PCA
#### Second Approach ===> Frequency Enconding, Oversampling and RFE

### Feature Enginering

In [ ]:
train['age'] = pd.cut(train['age'],5, labels = ['young', 'adult','grown','mayor','grandpa'])
test['age'] = pd.cut(test['age'],5, labels = ['young', 'adult','grown','mayor','grandpa'])

In [ ]:
# Get categorical variables in one DF
categorical_df = train.select_dtypes(exclude=[np.number])

In [ ]:
#Spliting Train into Train and Validation
X_train, X_test, y_train, y_test = train_test_split(train, train["subscribe"], test_size=0.30, random_state=42)
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(train, train["subscribe"], test_size=0.30, random_state=42)

test_2 = test

#### Firt Approach 

In [ ]:
X_train = pd.get_dummies(X_train, drop_first=True)
X_test = pd.get_dummies(X_test, drop_first=True)

test = pd.get_dummies(test, drop_first=True)

#### Second Approach

In [ ]:
# ---- Second Aprroach -------
### FREQUENCY ENCODING

### Econding tran set
for i in categorical_df.columns:
    # size of each category
    encoding = X_train_2.groupby(i).size()
    # get frequency of each category
    encoding = encoding/len(X_train_2)
    X_train_2[i] = X_train_2[i].map(encoding)


### enconding the validation set
for i in categorical_df.columns:
    # size of each category
    encoding = X_test_2.groupby(i).size()
    # get frequency of each category
    encoding = encoding/len(X_test_2)
    X_test_2[i] = X_test_2[i].map(encoding)



### enconding test holdout set
for i in categorical_df.columns:
    # size of each category
    encoding = test_2.groupby(i).size()
    # get frequency of each category
    encoding = encoding/len(test_2)
    test_2[i] = test_2[i].map(encoding)


## Data processing

#### First Approach

In [ ]:
# ************  OverSampling  ****************

#OverSampling with SMOTE
method = SMOTE(sampling_strategy=0.4) # making 60-40 difference
# Apply resampling to the training data only
X_resampled, y_resampled = method.fit_sample(X_train, y_train)

#Changing variable names back 
names = X_train.columns
X_train = pd.DataFrame(X_resampled, columns = names)
y_train = pd.DataFrame(y_resampled, columns = ['subscribe'])

In [ ]:
#Drop target and id 
#Train and Validation
X_train = X_train.drop({'subscribe', 'client_id'},axis =1)
X_test = X_test.drop({'subscribe', 'client_id'},axis =1)

#Test Holdout
client_id = pd.DataFrame(test['client_id']) # Save client_ID to indentify the predictions at the end
test = test.drop(['client_id'],axis=1) #Drop Client_id

In [ ]:
# **********  Scaling  ***************  

names = X_train.columns
X_train =  pd.DataFrame(scale(X_train), columns = names)
X_test =  pd.DataFrame(scale(X_test), columns = names)

# #Houldout 
names_test = test.columns
test =  pd.DataFrame(scale(test), columns = names_test)

In [ ]:
#**********  FEATURE REDUCTION  ***********
#PCA
#From 54 variables to 3 

#X_train
pca = PCA(n_components=3)
pca_data = pca.fit_transform(X_train)
X_train = pd.DataFrame(data = pca_data, columns = ['var1', 'var2','var3'])

#Validation
pca_data_val = pca.fit_transform(X_test)
X_test = pd.DataFrame(data = pca_data_val, columns = ['var1', 'var2','var3'])

#Test holdout
pca_data_test = pca.fit_transform(test)
test = pd.DataFrame(data = pca_data_test, columns = ['var1', 'var2','var3'])

#### Second Approach

In [ ]:
# ************  OverSampling  ****************

# Apply resampling to the training data only
X_resampled_2, y_resampled_2 = method.fit_sample(X_train_2, y_train_2)

#Changing variable names back 
names = X_train_2.columns
X_train_2 = pd.DataFrame(X_resampled_2, columns = names)
y_train_2 = pd.DataFrame(y_resampled_2, columns = ['subscribe'])

In [ ]:
#Drop target and id 
#Train and Validation
X_train_2 = X_train_2.drop({'subscribe', 'client_id'},axis =1)
X_test_2 = X_test_2.drop({'subscribe', 'client_id'},axis =1)

#Test Holdout
client_id = pd.DataFrame(test_2['client_id']) # Save client_ID to indentify the predictions at the end
test_2 = test_2.drop(['client_id'],axis=1) #Drop Client_id

In [ ]:
from sklearn.feature_selection import RFE

In [ ]:
#**********  FEATURE REDUCTION  ***********

#This method uses the feature importance of the model (estimator). In this case is the feature importances that RF gives to all the variables. If it was Logistic Regression it would be the coefficient. 
rfc=RandomForestClassifier()
# Assign Feature Selection method
rfe_selector = RFE(estimator= rfc, n_features_to_select=15, step=10, verbose=5)
# Fit it and get best X freatures
rfe_selector.fit(X_train_2, y_train_2)

rfe_support = rfe_selector.get_support()
rfe_feature = X_train_2.loc[:,rfe_support].columns.tolist()
print(str(len(rfe_feature)), 'selected features')

X_train_2 = X_train_2[rfe_feature] 

# Get same columns in validation test
X_test_2 = X_test_2[rfe_feature]

#Get same columns in test holdout 
test_2 = test_2[rfe_feature]

## Modeling

In [ ]:
#************  Models & Parameters   *************

# --- Tree based models ----

# Decision Tree
tree = DecisionTreeClassifier()
treeParam = {'criterion' : ['gini', 'entropy'], 'max_depth' : [4, 6, 8]}
#Ensemble

# Baggin
bag = BaggingClassifier()
bagParam = {'n_estimators': [100, 200, 500, 1000], 'max_depth' : [4, 6, 8]}

#Random Forest
rfc=RandomForestClassifier()
rfcParam = {'n_estimators': [100, 200, 500, 1000],'max_features': ['log2', 'sqrt'],'max_depth' : [4, 6, 8]}


# Boosting
#Adaboost
adab = AdaBoostClassifier()
adabParam = {'n_estimators': [100, 200, 500, 1000], 'learning_rate': [0.1, 0.05, 0.01, 0.001],'max_depth' : [5,15,25]}

#Gradient Boosting
gb = GradientBoostingClassifier()
gbParam = {'n_estimators': [100, 200, 500, 1000], 'max_features': ['log2', 'sqrt'],'learning_rate': [0.1, 0.05, 0.01, 0.001], 'max_depth': [4, 6, 8]}




#KNN
knn = KNeighborsClassifier()
knnParams = {'n_neighbors':[4,5,7],'algorithm':['auto', 'kd_tree']}

#Logistic Regression
lr = LogisticRegression()
c_space = np.logspace(-5, 8, 15, 20)
lrParams = {'C': c_space, 'penalty' : ['l1', 'l2']}


#Support Vector Machine
svm = SVC()
svmParams = {'C': [0.1, 10], 'gamma': [1,0.1],'kernel': ['poly', 'sigmoid']}

# MLP Classifier
mlp = MLPClassifier()
mlpParams = {'max_iter': [10],  'hidden_layer_sizes':np.arange(10, 15), 'random_state':[0,1]}

#### Tree, Bagging and RF

In [ ]:
#************  Models & Parameters   *************

# --- Tree based models ----

# Decision Tree
tree = DecisionTreeClassifier()
treeParam = {'criterion' : ['gini', 'entropy'], 'max_depth' : [4, 6, 8]}
#Ensemble

# Baggin
bag = BaggingClassifier()
bagParam = {'n_estimators': [100, 200, 500, 1000]}

#Random Forest
rfc=RandomForestClassifier()
rfcParam = {'n_estimators': [100, 200, 500, 1000],'max_features': ['log2', 'sqrt'],'max_depth' : [4, 6, 8]}


In [ ]:
# ********************* Hyperparameter Tunning ******************

random.seed(30)
#list of models for loop
models = (tree, bag, rfc)
models2 = ('Tree', 'Baggin','Random Forest')
#list of param for loop
param = (treeParam, bagParam, rfcParam)

#List to append results
auc = []
accuracy = []
modelparam= []
recall = []
precision = []
f1 = []

for i in range(len(models)):
    #Loop list of models and its parameters
    cv = GridSearchCV(models[i], param[i], cv=4)
    #Train model with train set
    cvModel = cv.fit(X_train, y_train)
    
    #Train:
    #Get probabilitie predictions on train
    preds_train = cvModel.predict_proba(X_train)[:,1]
    #Get numeric prediction on tran
    predi_num_t = cvModel.predict(X_train)
    
    #Test
    #Get probability predictions on test
    preds = cvModel.predict_proba(X_test)[:,1]
    #Get numeric prediction on test
    predi_num = cvModel.predict(X_test)
    
    
    #Save best model
    #save predictions of number of train
    #save probability predictions of Train
    #save predictions of number of test
    #save probability predictions of Test
    if i == 0: 
        model1 = cvModel
        pred1_num_t = predi_num_t
        pred1_t = preds_train
        pred1_num = predi_num
        pred1 = preds
    elif i == 1:
        model2 = cvModel
        pred2_num_t = predi_num_t
        pred2_t = preds_train
        pred2_num = predi_num
        pred2 = preds
    elif i == 2: 
        model3 = cvModel
        pred3_num_t = predi_num_t
        pred3_t = preds_train
        pred3_num = predi_num
        pred3 = preds
    elif i == 3: 
        model4 = cvModel
        pred4_num_t = predi_num_t
        pred4_t = preds_train
        pred4_num = predi_num 
        pred4 = preds
    else:
        model5 = cvModel
        pred5_num_t = predi_num_t
        pred5_t = preds_train
        pred5_num = predi_num
        pred5 = preds

    #METRICS
    #AUC
    test_roc_auc = roc_auc_score(y_test, preds)
    auc.append([roc_auc_score(y_test, preds)])
    #Accuracy
    accuracy.append([cvModel.score(X_test, y_test)])
    #Parameters best model
    modelparam.append(cvModel.best_estimator_)
    #Classification Report (precision    recall  f1-score   support)
    recall.append([recall_score(y_test,predi_num)])
    precision.append([precision_score(y_test,predi_num)])
    f1.append([f1_score(y_test,predi_num)])
    print(models2[i])
        
#Creating Dataframe Metrics
metrics = pd.DataFrame(list(zip(models2,auc, accuracy, recall, precision,f1, modelparam )), columns =['Models', 'AUC','Accuracy','Recall','Precision','F1','Param']) 

#Round and Numeric variables
for x in metrics.iloc[:,1:6].columns:
     metrics[x] = metrics[x].astype(str)
     metrics[x] = metrics[x].str.replace("]"," ")
     metrics[x] = metrics[x].str.replace("["," ")
     metrics[x] = metrics[x].astype(float)
     metrics[x] = round(metrics[x],3)
metrics

#### Boosting 

In [ ]:
# Boosting
#Adaboost
adab = AdaBoostClassifier()
adabParam = {'n_estimators': [100, 500, 1000], 'learning_rate': [0.1, 0.01, 0.001]}

#Gradient Boosting
gbm = GradientBoostingClassifier()
gbmParam = {'n_estimators': [100, 500, 1000], 'learning_rate': [0.1, 0.05, 0.01, 0.001], 'max_depth': [4, 6, 8]}


In [ ]:
# ********************* Hyperparameter Tunning ******************

random.seed(30)
#list of models for loop
models = (adab, gbm)
models2 = ('AdaBoost', 'GBM')
#list of param for loop
param = (adabParam, gbmParam)

#List to append results
auc = []
accuracy = []
modelparam= []
recall = []
precision = []
f1 = []

for i in range(len(models)):
    #Loop list of models and its parameters
    cv = GridSearchCV(models[i], param[i], cv=4)
    #Train model with train set
    cvModel = cv.fit(X_train, y_train)
    
    #Train:
    #Get probabilitie predictions on train
    preds_train = cvModel.predict_proba(X_train)[:,1]
    #Get numeric prediction on tran
    predi_num_t = cvModel.predict(X_train)
    
    #Test
    #Get probability predictions on test
    preds = cvModel.predict_proba(X_test)[:,1]
    #Get numeric prediction on test
    predi_num = cvModel.predict(X_test)
    
    
    #Save best model
    #save predictions of number of train
    #save probability predictions of Train
    #save predictions of number of test
    #save probability predictions of Test
    if i == 0: 
        model1 = cvModel
        pred1_num_t = predi_num_t
        pred1_t = preds_train
        pred1_num = predi_num
        pred1 = preds
    elif i == 1:
        model2 = cvModel
        pred2_num_t = predi_num_t
        pred2_t = preds_train
        pred2_num = predi_num
        pred2 = preds
    elif i == 2: 
        model3 = cvModel
        pred3_num_t = predi_num_t
        pred3_t = preds_train
        pred3_num = predi_num
        pred3 = preds
    elif i == 3: 
        model4 = cvModel
        pred4_num_t = predi_num_t
        pred4_t = preds_train
        pred4_num = predi_num 
        pred4 = preds
    else:
        model5 = cvModel
        pred5_num_t = predi_num_t
        pred5_t = preds_train
        pred5_num = predi_num
        pred5 = preds

    #METRICS
    #AUC
    test_roc_auc = roc_auc_score(y_test, preds)
    auc.append([roc_auc_score(y_test, preds)])
    #Accuracy
    accuracy.append([cvModel.score(X_test, y_test)])
    #Parameters best model
    modelparam.append(cvModel.best_estimator_)
    #Classification Report (precision    recall  f1-score   support)
    recall.append([recall_score(y_test,predi_num)])
    precision.append([precision_score(y_test,predi_num)])
    f1.append([f1_score(y_test,predi_num)])
    print(models2[i])
        
#Creating Dataframe Metrics
metrics = pd.DataFrame(list(zip(models2,auc, accuracy, recall, precision,f1, modelparam )), columns =['Models', 'AUC','Accuracy','Recall','Precision','F1','Param']) 

#Round and Numeric variables
for x in metrics.iloc[:,1:6].columns:
     metrics[x] = metrics[x].astype(str)
     metrics[x] = metrics[x].str.replace("]"," ")
     metrics[x] = metrics[x].str.replace("["," ")
     metrics[x] = metrics[x].astype(float)
     metrics[x] = round(metrics[x],3)
metrics

## Visualization of Results 

In [ ]:
##PLOT of TRAIN and TEST ROC RF (second best performer)

# Function
def plot_train_test(y_test, pred_test, y_train, pred_train ):
    fpr, tpr, thresholds = roc_curve(y_test,pred_test)
    fpr2, tpr2, thresholds = roc_curve(y_train, pred_train)

    # Plot ROC curve
    plt.plot([0, 1], [0, 1], 'k--')
    plt.plot(fpr, tpr, label= "Test")
    plt.plot([0, 1], [0, 1], 'k--')
    plt.plot(fpr2, tpr2, label = "Train", color='orange')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve RF')
    plt.legend()
    return plt.show()

In [ ]:
#AdaBoost
plot_train_test(y_test, pred1, y_train, pred1_t)

In [ ]:
#Gradient Boosting
plot_train_test(y_test, pred2, y_train, pred2_t)

In [ ]:
#Random Forest
plot_train_test(y_test, pred3, y_train, pred3_t)

In [ ]:
#PLOT of BENCHMARK TEST ROC CURVS

# Generate ROC curve values: fpr, tpr, thresholds
fpr, tpr, thresholds = roc_curve(y_test,pred1)
b, bb, thresholds = roc_curve(y_test,pred2)
c, cc, thresholds = roc_curve(y_test,pred3)
#d, dd, thresholds = roc_curve(y_test,pred4)

# Plot ROC curve
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr, tpr, label= "AdaBoost")
plt.plot(b, bb, label = "GBM", color='orange')
plt.plot(c, cc, label = "RF", color='red')
#plt.plot(d, dd, label = "KNN", color='green')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve Test Sets')
plt.legend()
plt.show()

## Second Approach

In [ ]:
#************  Models & Parameters   *************

# --- Tree based models ----

# Decision Tree
tree = DecisionTreeClassifier()
treeParam = {'criterion' : ['gini', 'entropy'], 'max_depth' : [4, 6, 8]}
#Ensemble

# Baggin
bag = BaggingClassifier()
bagParam = {'n_estimators': [100, 200, 500, 1000]}

#Random Forest
rfc=RandomForestClassifier()
rfcParam = {'n_estimators': [100, 200, 500, 1000],'max_features': ['log2', 'sqrt'],'max_depth' : [4, 6, 8]}


In [ ]:
# ********************* Hyperparameter Tunning ******************

random.seed(30)
#list of models for loop
models = (tree, bag, rfc)
models2 = ('Tree', 'Baggin','Random Forest')
#list of param for loop
param = (treeParam, bagParam, rfcParam)

#List to append results
auc = []
accuracy = []
modelparam= []
recall = []
precision = []
f1 = []

for i in range(len(models)):
    #Loop list of models and its parameters
    cv = GridSearchCV(models[i], param[i], cv=4)
    #Train model with train set
    cvModel = cv.fit(X_train_2, y_train_2)
    
    #Train:
    #Get probabilitie predictions on train
    preds_train = cvModel.predict_proba(X_train_2)[:,1]
    #Get numeric prediction on tran
    predi_num_t = cvModel.predict(X_train_2)
    
    #Test
    #Get probability predictions on test
    preds = cvModel.predict_proba(X_test_2)[:,1]
    #Get numeric prediction on test
    predi_num = cvModel.predict(X_test_2)
    
    
    #Save best model
    #save predictions of number of train
    #save probability predictions of Train
    #save predictions of number of test
    #save probability predictions of Test
    if i == 0: 
        model1 = cvModel
        pred1_num_t = predi_num_t
        pred1_t = preds_train
        pred1_num = predi_num
        pred1 = preds
    elif i == 1:
        model2 = cvModel
        pred2_num_t = predi_num_t
        pred2_t = preds_train
        pred2_num = predi_num
        pred2 = preds
    elif i == 2: 
        model3 = cvModel
        pred3_num_t = predi_num_t
        pred3_t = preds_train
        pred3_num = predi_num
        pred3 = preds
    elif i == 3: 
        model4 = cvModel
        pred4_num_t = predi_num_t
        pred4_t = preds_train
        pred4_num = predi_num 
        pred4 = preds
    else:
        model5 = cvModel
        pred5_num_t = predi_num_t
        pred5_t = preds_train
        pred5_num = predi_num
        pred5 = preds

    #METRICS
    #AUC
    test_roc_auc = roc_auc_score(y_test_2, preds)
    auc.append([roc_auc_score(y_test_2, preds)])
    #Accuracy
    accuracy.append([cvModel.score(X_test_2, y_test)])
    #Parameters best model
    modelparam.append(cvModel.best_estimator_)
    #Classification Report (precision    recall  f1-score   support)
    recall.append([recall_score(y_test_2,predi_num)])
    precision.append([precision_score(y_test_2,predi_num)])
    f1.append([f1_score(y_test_2,predi_num)])
    print(models2[i])
        
#Creating Dataframe Metrics
metrics = pd.DataFrame(list(zip(models2,auc, accuracy, recall, precision,f1, modelparam )), columns =['Models', 'AUC','Accuracy','Recall','Precision','F1','Param']) 

#Round and Numeric variables
for x in metrics.iloc[:,1:6].columns:
     metrics[x] = metrics[x].astype(str)
     metrics[x] = metrics[x].str.replace("]"," ")
     metrics[x] = metrics[x].str.replace("["," ")
     metrics[x] = metrics[x].astype(float)
     metrics[x] = round(metrics[x],3)
metrics

In [ ]:
metrics.to_csv('C:/Users/aolivera/OneDrive - IESEG/IESEG/Courses/Second Semester/SL&ML/Individual Assignment/RANDOM.csv', index = False)   

In [ ]:
# Boosting
#Adaboost
adab = AdaBoostClassifier()
adabParam = {'n_estimators': [100, 500, 1000], 'learning_rate': [0.1,  0.01, 0.001]}

#Gradient Boosting
gbm = GradientBoostingClassifier()
gbmParam = {'n_estimators': [100, 200, 500], 'max_features': ['log2', 'sqrt'],'learning_rate': [0.1, 0.01, 0.001], 'max_depth': [4, 6, 8]}


In [ ]:
# ********************* Hyperparameter Tunning ******************

random.seed(30)
#list of models for loop
models = (adab, gbm)
models2 = ('AdaBoost', 'GBM')
#list of param for loop
param = (adabParam, gbmParam)

#List to append results
auc = []
accuracy = []
modelparam= []
recall = []
precision = []
f1 = []

for i in range(len(models)):
    #Loop list of models and its parameters
    cv = GridSearchCV(models[i], param[i], cv=4)
    #Train model with train set
    cvModel = cv.fit(X_train_2, y_train_2)
    
    #Train:
    #Get probabilitie predictions on train
    preds_train = cvModel.predict_proba(X_train_2)[:,1]
    #Get numeric prediction on tran
    predi_num_t = cvModel.predict(X_train_2)
    
    #Test
    #Get probability predictions on test
    preds = cvModel.predict_proba(X_test_2)[:,1]
    #Get numeric prediction on test
    predi_num = cvModel.predict(X_test_2)
    
    
    #Save best model
    #save predictions of number of train
    #save probability predictions of Train
    #save predictions of number of test
    #save probability predictions of Test
    if i == 0: 
        model1 = cvModel
        pred1_num_t = predi_num_t
        pred1_t = preds_train
        pred1_num = predi_num
        pred1 = preds
    elif i == 1:
        model2 = cvModel
        pred2_num_t = predi_num_t
        pred2_t = preds_train
        pred2_num = predi_num
        pred2 = preds
    elif i == 2: 
        model3 = cvModel
        pred3_num_t = predi_num_t
        pred3_t = preds_train
        pred3_num = predi_num
        pred3 = preds
    elif i == 3: 
        model4 = cvModel
        pred4_num_t = predi_num_t
        pred4_t = preds_train
        pred4_num = predi_num 
        pred4 = preds
    else:
        model5 = cvModel
        pred5_num_t = predi_num_t
        pred5_t = preds_train
        pred5_num = predi_num
        pred5 = preds

    #METRICS
    #AUC
    test_roc_auc = roc_auc_score(y_test_2, preds)
    auc.append([roc_auc_score(y_test_2, preds)])
    #Accuracy
    accuracy.append([cvModel.score(X_test_2, y_test)])
    #Parameters best model
    modelparam.append(cvModel.best_estimator_)
    #Classification Report (precision    recall  f1-score   support)
    recall.append([recall_score(y_test_2,predi_num)])
    precision.append([precision_score(y_test_2,predi_num)])
    f1.append([f1_score(y_test_2,predi_num)])
    print(models2[i])
        
#Creating Dataframe Metrics
metrics = pd.DataFrame(list(zip(models2,auc, accuracy, recall, precision,f1, modelparam )), columns =['Models', 'AUC','Accuracy','Recall','Precision','F1','Param']) 

#Round and Numeric variables
for x in metrics.iloc[:,1:6].columns:
     metrics[x] = metrics[x].astype(str)
     metrics[x] = metrics[x].str.replace("]"," ")
     metrics[x] = metrics[x].str.replace("["," ")
     metrics[x] = metrics[x].astype(float)
     metrics[x] = round(metrics[x],3)
metrics

In [ ]:
metrics.to_csv('C:/Users/aolivera/OneDrive - IESEG/IESEG/Courses/Second Semester/SL&ML/Individual Assignment/ada_gbm.csv', index = False)   

### Visualization of Results 2

In [ ]:
##PLOT of TRAIN and TEST ROC RF (second best performer)

# # Generate ROC curve values: fpr, tpr, thresholds
def plot_train_test(y_test, pred_test, y_train, pred_train ):
    fpr, tpr, thresholds = roc_curve(y_test,pred_test)
    fpr2, tpr2, thresholds = roc_curve(y_train, pred_train)

    # Plot ROC curve
    plt.plot([0, 1], [0, 1], 'k--')
    plt.plot(fpr, tpr, label= "Test")
    plt.plot([0, 1], [0, 1], 'k--')
    plt.plot(fpr2, tpr2, label = "Train", color='orange')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve RF')
    plt.legend()
    return plt.show()

In [ ]:
#AdaBoost
plot_train_test(y_test, pred1, y_train, pred1_t)

In [ ]:
#Gradient Boosting
plot_train_test(y_test, pred2, y_train, pred2_t)

In [ ]:
#PLOT of BENCHMARK TEST ROC CURVS

# Generate ROC curve values: fpr, tpr, thresholds
fpr, tpr, thresholds = roc_curve(y_test,pred1)
b, bb, thresholds = roc_curve(y_test,pred2)
c, cc, thresholds = roc_curve(y_test,pred3)
#d, dd, thresholds = roc_curve(y_test,pred4)

# Plot ROC curve
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr, tpr, label= "AdaBoost")
plt.plot(b, bb, label = "GBM", color='orange')
plt.plot(c, cc, label = "RF", color='red')
#plt.plot(d, dd, label = "KNN", color='green')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve Test Sets')
plt.legend()
plt.show()

### Predictions to Kaggle

In [ ]:
# *************  Method 1 ***************
# Predicting with best models
final_models = [model1, model2, model3, model4, model5]
predictions_df = ['m', 'n', 'o', "p", 'j']
for p in range(0,5):
    predictions_kaggle = final_models[p].predict_proba(test)[:,1]
    predictions_df[p] = pd.DataFrame(predictions_kaggle, columns = ['subscribe'])
    predictions_df[p]['client_id'] = client_id  

In [ ]:
#Downloading results
predictions_df[1].to_csv('C:/Users/aolivera/OneDrive - IESEG/IESEG/Courses/Second Semester/SL&ML/Kaggle/model_pca_smot10.csv', index = False)   

In [ ]:
# *************    Method 2   **********************

# All process again on full Train data
train = pd.read_csv('C:/Users/aolivera/OneDrive - IESEG/IESEG/Courses/Second Semester/SL&ML/Kaggle/bank_mkt_train.csv')
train = pd.get_dummies(train, drop_first=True)
y_train = train['subscribe']
#Drop target and id 
train = train.drop({'subscribe', 'client_id'},axis =1)
names = train.columns
#scale
train =  pd.DataFrame(scale(train), columns = names)
#pca
pca = PCA(n_components=2)
pca_data = pca.fit_transform(train)
train = pd.DataFrame(data = pca_data, columns = ['princ_comp_1', 'princ_comp_2'])

In [ ]:
# Fitting the best model with the selected hyperparameters with the original full Train Data and making predictions
final_models = [model1, model2, model3, model4, model5]
predictions_df = ['m', 'n', 'o', "p", 'j']
for p in range(0,5):
    final_models[p] = final_models[p].fit(train,y_train)
    predictions_kaggle = final_models[p].predict_proba(test)[:,1]
    predictions_df[p] = pd.DataFrame(predictions_kaggle, columns = ['subscribe'])
    predictions_df[p]['client_id'] = client_id  

In [ ]:
#Downloading results
predictions_df[1].to_csv('C:/Users/aolivera/OneDrive - IESEG/IESEG/Courses/Second Semester/SL&ML/Kaggle/model_pca_smot10.csv', index = False)   